# Create adatom defects in a slab material

Create an adatom by specifying the chemical element, approximate position on surface and distance z, as well as placement method which will be resolved to:
- exact coordinate,
- closest site to provided coordinate,
- the equidistant position between the closes atoms on the surface according to Voronoi tesselation, 
- or the crystal site of the next layer that is closest to specified position.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set defects parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Defect: Add an adatom defect to the slab material
2. Visualize the Defect: Visualize the defect structure

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
ELEMENT = "Si"  # Chemical element of the adatom
DEFECT_CONFIGS = [
    {
        "type": "adatom",
        "coordinate": [0.5, 0.5], # Crystal coordinates on the surface (x, y)
        "distance_z": 1.0,  # Method to place the adatom
        "element": ELEMENT,
    }
]


PLACEMENT_METHOD = "new_crystal_site"  # Method to place the adatom, e.g., "NEW_CRYSTAL_SITE", "EQUIDISTANT", "EXACT_COORDINATE"


# Slab parameters
MILLER_INDICES = (1, 1, 1)  # Miller indices of the surface
SLAB_THICKNESS = 3  # Thickness of the slab in unit cells
VACUUM = 5.0  # Vacuum thickness in Angstrom
XY_SUPERCELL_MATRIX = [[2, 0], [0, 2]]  # Supercell matrix for the slab
TERMINATION_FORMULA = None  # Stoichiometric formula of the slab termination to be used.

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gd925f84-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils', deps=False)

    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("create_point_defect.ipynb")

### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Slab

In [ ]:
from mat3ra.made.tools.analyze.lattice_planes import CrystalLatticePlanesMaterialAnalyzer
from mat3ra.made.tools.build.slab.helpers import create_slab, get_slab_terminations
from mat3ra.made.tools.build.slab.termination_utils import select_slab_termination
from utils.visualize import visualize_materials as visualize

material = materials[0]
# Create analyzer to get terminations
analyzer = CrystalLatticePlanesMaterialAnalyzer(material=material, miller_indices=MILLER_INDICES)
slab_terminations = analyzer.terminations
# Get termination from analyzer
terminations = get_slab_terminations(material, MILLER_INDICES)
termination = select_slab_termination(terminations, TERMINATION_FORMULA)

slab = create_slab(
    crystal=material,
    termination=termination,
    miller_indices=MILLER_INDICES,
    number_of_layers=SLAB_THICKNESS,
    vacuum=VACUUM,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_c=True,
    use_conventional_cell=True
)
visualize([{"material": slab, "rotation": "0x"}, {"material": slab, "rotation": "-90x"}], repetitions=[1, 1, 1])

## 2. Create the Defect

In [ ]:
from mat3ra.made.tools.build.defect.adatom.helpers import create_multiple_adatom_defects

slab_with_adatom = create_multiple_adatom_defects(
    slab=slab,
    defect_dicts=DEFECT_CONFIGS,
    placement_method=PLACEMENT_METHOD,
)

## 3. Visualize the Slab with Adatom

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([
    {"material": slab, "title": "Original material"},
    {"material": slab_with_adatom, "title": f"Material with adatom defects"}
],
    viewer="wave"
)

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials([slab_with_adatom])